---

# CSCI 3202, Spring 2022
# Final Project

<br> 

### Your name: Felipe Lima

<br> 

---

For my final project, I was decided to implement a Bayesian spam filtering. The idea was to create a spam filter to detect unsolicited and unwanted emails using Naive Bayes Theorem to calculate whether the message is spam or not. The system calculates the probability of a certain message being spam based on words in the title and message, learning from messages that were identified as spam and messages that were identified as not being spam. The scope of the project is based on a pre-defined data set and applying the algorithm to it would separate unwanted messages and normal email.

**Importing** some useful packages and libraries:

In [1]:
import pandas as pd
import re
import random

**Reading dataset** containing a collection of sms messages with SPAM and HAM messages:

Dataset source: https://archive-beta.ics.uci.edu/ml/datasets/sms+spam+collection

In [2]:
spam_or_ham = pd.read_csv('spam_ham_data', sep='\t',
names=['Type', 'Msg'])
print("Total messages:")
print(spam_or_ham.shape[0])
print("Count: ")
print(spam_or_ham.groupby('Type').count())
print("------")
print("Percentage: ")
print(spam_or_ham['Type'].value_counts(normalize=True) *100)
spam_or_ham.head()

Total messages:
5572
Count: 
       Msg
Type      
ham   4825
spam   747
------
Percentage: 
ham     86.593683
spam    13.406317
Name: Type, dtype: float64


,Type,Msg
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


We can observe that in the imported data set there are a total of 5572 messages from which 4825 (86.59%) are normal messages (ham) and 747 (13.41%) are spam. 

For the purpose of this assignment, ponctuations and non alphanumeric characters would just get in the way of proper working of the code so the next step was to **clean** the data set and set all characters to lower case so the comparison of words work the right way.

In [3]:
spam_or_ham['Msg'] = spam_or_ham['Msg'].str.replace('\W+', ' ').str.replace('\s+', ' ').str.strip() #replaces any non alphanummeric character with a space, replaces any extra spaces with a single space and strips the string fro newlines and straineous characters
spam_or_ham['Msg'] = spam_or_ham['Msg'].str.lower() #transfors all charactes into lower case
spam_or_ham['Msg'] = spam_or_ham['Msg'].str.split() #splits the string into comma separated items on a list
spam_or_ham.head()

,Type,Msg
0,ham,"[go, until, jurong, point, crazy, available, o..."
1,ham,"[ok, lar, joking, wif, u, oni]"
2,spam,"[free, entry, in, 2, a, wkly, comp, to, win, f..."
3,ham,"[u, dun, say, so, early, hor, u, c, already, t..."
4,ham,"[nah, i, don, t, think, he, goes, to, usf, he,..."


Now the column "Msg" from the data set consists of a list of comma separated strings which will be easier to capture individual words and count the occurrences.

Next I **split** the dataset into a training set and a testing set. The training set is used so code learns what words are usually contained in spam emails and which ones are contained in normal messages. The testing set is used to calculate how well the algorithm did. To do so, first **randomizing** the full set guarantees that the training set and the test set contains roughly the same percentage of ham and spam. 

In [4]:
train_set = spam_or_ham.sample(frac=0.8,random_state=1).reset_index(drop=True) #ramdomizes data set and gets 80% of it
test_set = spam_or_ham.drop(train_set.index).reset_index(drop=True) #rest of the data set
train_set = train_set.reset_index(drop=True)  #resets index
print("Total messages:")
print(spam_or_ham.shape[0])
print("Count: ")
print(spam_or_ham.groupby('Type').count())
print("Percentage: ")
print(spam_or_ham['Type'].value_counts(normalize=True) *100)
print("------")
print("Training set:")
print("Total messages:")
print(train_set.shape[0])
print("Count: ")
print(train_set.groupby('Type').count())
print("Percentage: ")
print(train_set['Type'].value_counts(normalize=True) *100)
print("------")
print("Total messages:")
print(test_set.shape[0])
print("Test set:")
print("Count: ")
print(test_set.groupby('Type').count())
print("Percentage: ")
print(test_set['Type'].value_counts(normalize=True) *100)

Total messages:
5572
Count: 
       Msg
Type      
ham   4825
spam   747
Percentage: 
ham     86.593683
spam    13.406317
Name: Type, dtype: float64
------
Training set:
Total messages:
4458
Count: 
       Msg
Type      
ham   3858
spam   600
Percentage: 
ham     86.54105
spam    13.45895
Name: Type, dtype: float64
------
Total messages:
1114
Test set:
Count: 
      Msg
Type     
ham   969
spam  145
Percentage: 
ham     86.983842
spam    13.016158
Name: Type, dtype: float64


After spliting the original data set into training and test, we have around the same percentage of ham (\~87%) and spam (\~13%) in all of them. The original set containing all the messages, the training set containing approximately 80% of the orignal set and the test set containing approximately 20% of the original set.

Next I **created a list** of all unique words in all of the messages from the training set.

In [5]:
vocab = train_set['Msg'].sum() #creates a list of all the words in the set
vocab = set(vocab) #removes duplicate words from the list and make the list a set
vocab = list(vocab) #reverts vocab to a list

Then I **created a dictionary** of all the words and the number of times they appear on the training set and made it into a data set that is then concatenated with the orginal training data set so we have the number of times each unique word appear on each unique message.

In [6]:
#creates a dictionary of key being a unique word from vocab and entries composed of a list of size equal to the lenght of the set
word_msg ={}
for word in vocab:
    word_msg.update({word: [0] * len(train_set['Msg'])})
for i in range(len(train_set['Msg'])): #loops over the messages on the train set
    for word in train_set['Msg'][i]: #loops over each word on the list of each Msg on the training set
        word_msg[word][i] += 1 #increments the count of the word "word" on each "Msg" the "word" appears

words_df = pd.DataFrame(word_msg) #make it into a dataframe and concatenates with training set
train_set = pd.concat([train_set, words_df], axis=1) #concatenates with training set
train_set

,Type,Msg,ummifying,2wu,guessed,thread,hospital,pushbutton,needle,message,...,intro,smoking,july,withdraw,shake,tor,spotty,needing,gt,mumhas
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4453,ham,"[sorry, i, ll, call, later, in, meeting, any, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4454,ham,"[babe, i, fucking, love, you, too, you, know, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4455,spam,"[u, ve, been, selected, to, stay, in, 1, of, 2...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4456,ham,"[hello, my, boytoy, geeee, i, miss, you, alrea...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


From the table we can see there are 7783 unique words in the data set. Since a lot of them are very specific the majority of the words are listed as 0 appearences as they only appear on a selected number of messages and due to the number of columns and rows not all of the words and messages are listed on the table above.

Now the data set that is going to be used to train the code is organized and clean I will move on the **calculations**.

In order to calculate whether a specific message is spam or ham, we want to compare probabilities of the message being spam/ham given the content (text) of the message. Meaning:

$$P(\text{spam/ham}|w_1, w_2, w_3 ... w_n)$$

Because we are applying Naive Bayes Theorem to the equation and we calculated the probability of discrete, individual words and not the probabilities of something continuous these probabilities are also called likelihoods. So we have that:

$$P(\text{spam/ham}|w_1, w_2, w_3 ... w_n) \propto P(\text{spam/ham}) \cdot P(w_1|\text{spam/ham}) \cdot P(w_2|\text{spam/ham}) \cdot P(w_3|\text{spam/ham}) \cdot ... \cdot P(w_n|\text{spam/ham})$$

and therefore:

$$P(\text{spam/ham}|w_1, w_2, w_3 ... w_n) \propto P(\text{spam/ham}) \cdot \prod_{i = 1}^{n} P(w_i|\text{spam/ham})$$

Great source to understand the calculations: https://www.youtube.com/watch?v=O2L2Uv9pdDA

In the spirit of facilitating the understanding of the code and reducing the number of calculations I **stored the constant values** in variables. The following code contains:
- A data frame of spam and ham messages separetly
- Probability of a message being ham or spam
- The number of words in spam and ham messages separetly
- The length of the vocab list

In [7]:
spam_msgs = train_set[train_set['Type'] == 'spam'] #df of all spam messages
ham_msgs = train_set[train_set['Type'] == 'ham'] #df of all ham messages

p_ham = train_set['Type'].value_counts(normalize=True)[0] #probability of a message being ham
p_spam = train_set['Type'].value_counts(normalize=True)[1] #probability of a message being spam

nw_spam = len(spam_msgs['Msg'].sum()) #number of words in all the spam messages
nw_ham = len(ham_msgs['Msg'].sum()) #number of words in all the ham messages

len_vocab = len(vocab) #number of words in the vocab list

Now that constant values were stored, I'll **create helper functions** to calculate $P(w|spam)$ and $P(w|ham)$.

In [8]:
alpha = 1 #Laplace smoothing
def pw_spam(word):
    return (spam_msgs[word].sum() + alpha)  / (nw_spam + alpha*len_vocab)
    
def pw_ham(word):
    return (ham_msgs[word].sum() + alpha)  / (nw_ham + alpha*len_vocab)

In order to calculate the above probabilities the formula used was the following:
    $$P(w|spam/ham) = \frac{\text{# of times a word "w" occurs in in spam/ham messages} + \alpha}{\text{# of words in spam/ham messages} + \alpha*\text{total number of words in vocab}}$$
    
This accounts for the probability of a word given it is a spam/ham message + Laplace smoothing

Lastly the identify function below is reponsible for calculating the probability of a message being spam or ham and comparing both to then identify whether the message is spam or ham. The greater probability is the classification of the message.

The function calculates the probability following the formula:

$$P(\text{spam/ham}|w_1, w_2, w_3 ... w_n) \propto P(\text{spam/ham}) \cdot \prod_{i = 1}^{n} P(w_i|\text{spam/ham})$$

**Note** - if a word passed to the identify function is not present in the trained data set the function will just ignore it. The hope is that the data set is large enough to account for most words in the english language.

In [9]:
def identify(msg, flag):
    msg = re.sub('\W', ' ', msg).lower().split() #clean message
    p_msg_spam = 1
    p_msg_ham = 1
    for word in msg: #for every word in the message passed as input
        if word in train_set.columns: #if the word is in the data set
            p_msg_spam *= pw_spam(word) #multiply the prior probability to P(word|spam)
            p_msg_ham *= pw_ham(word) #multiply the prior probability to P(word|ham)
    
    p_msg_spam *= p_spam # multiplies by the probability of spam
    p_msg_ham *= p_ham # multiplies by the probability of spam
    if (flag == True):
        print('Probability of spam:', p_msg_spam)
        print('Probability of ham:', p_msg_ham)

    #compare probabilities to determine if a message is spam or ham
    if p_msg_ham > p_msg_spam:
        return 'ham'
    elif p_msg_ham < p_msg_spam:
        return 'spam'
    else:
        return 'equal'
    
def classi(classification):
    if (classification == 'ham'):
        print('The inputed message is: Ham')
    elif (classification == 'spam'):
        print('The inputed message is: Spam')
    else:
        print('Equal proabilities, human needed to identify message')

Testing for a user input. In this case the email sent with this project proposal:

In [10]:
user_input = "For my final project, I was thinking of doing Bayesian spam filtering. The idea is to create a spam filter to detect unsolicited and unwanted emails using a bayesian system to calculate whether the message is spam or not. The system would calculate a probability of whether a certain message is spam based on words in the title and message, learning from messages that were identified as spam and messages that were identified as not being spam. The scope of the project would have to be based on a pre-defined data set and applying the algorithm to it would separate unwanted messages and normal email.Hope to hear from you soon whether this project is approved or not!"
result = identify(user_input, True)
classi(result)

Probability of spam: 3.6782119079708777e-303
Probability of ham: 2.5944121027711605e-285
The inputed message is: Ham


Now let's test with a larger number of inputs to test for accuracy of the algorithm. Using the test data set with 20% of the origial set

In [11]:
counter = 0

for msg_i in range(len(test_set)): 
    if test_set['Type'][msg_i] == identify(str(test_set['Msg'][msg_i]), False):
        counter += 1
    
print('Correct classification:', counter)
print('Incorrect classification:', len(test_set) - counter)
print('Accuracy:', counter/len(test_set)*100, "%")

Correct classification: 1104
Incorrect classification: 10
Accuracy: 99.10233393177738 %


**Notes on performance**

As seen aboeve the filter had an accuracy of 99.1%, which is a promising result.

Here you can test your own input. Just run the next cell and input your message!

**Try your own input**

Just run the cell bellow and see if your input is classified as ham or spam!

In [12]:
input_message = input()
classi(identify(input_message, True))

Try your own input here!
Probability of spam: 3.585454711237706e-15
Probability of ham: 5.847714655437426e-13
The inputed message is: Ham


 **Notes on credit**
 
In order to create this spam filter I based my project on different websites implementing a similar technique but typing explanations and code on my own. I did my best to optmize the code and I managed to get better results than the ones from websites I got my inspiration from. Here is a list of websites used:

- https://archive-beta.ics.uci.edu/ml/datasets/sms+spam+collection
- https://www.kdnuggets.com/2020/07/spam-filter-python-naive-bayes-scratch.html
- https://www.youtube.com/watch?v=O2L2Uv9pdDA
- https://towardsdatascience.com/logic-and-implementation-of-a-spam-filter-machine-learning-algorithm-a508fb9547bd
- https://towardsdatascience.com/laplace-smoothing-in-na%C3%AFve-bayes-algorithm-9c237a8bdece#:~:text=Laplace%20smoothing%20is%20a%20smoothing%20technique%20that%20helps%20tackle%20the,the%20positive%20and%20negative%20reviews
